In [1]:
import requests
from urllib.parse import quote

In [2]:
ship = "C8X Pisces Expedition"

In [9]:
data = requests.get(
    f"https://api.star-citizen.wiki/api/v2/vehicles/{quote(ship)}",
    params={
        "locale": "en_EN",
        "include": "components,hardpoints,shops"
    }
).json()

In [10]:
data

{'data': {'uuid': '831fdcfc-337a-4350-967d-e693d19efeee',
  'name': 'C8X Pisces Expedition',
  'slug': 'c8x-pisces-expedition',
  'class_name': 'ANVL_C8X_Pisces_Expedition',
  'sizes': {'length': 16, 'beam': 12, 'height': 8},
  'emission': {'ir': 6034, 'em_idle': 4859, 'em_max': 20278},
  'mass': 48143,
  'cargo_capacity': 4,
  'vehicle_inventory': 4,
  'personal_inventory': 0,
  'crew': {'min': 1, 'max': None, 'weapon': 0, 'operation': None},
  'health': 3250,
  'shield_hp': 1500,
  'speed': {'scm': 142,
   'max': 1150,
   'zero_to_scm': 2.805,
   'zero_to_max': 22.717,
   'scm_to_zero': 28.979,
   'max_to_zero': 234.693},
  'fuel': {'capacity': 522500,
   'intake_rate': 12,
   'usage': {'main': 1, 'maneuvering': 14, 'retro': 0, 'vtol': 0}},
  'quantum': {'quantum_speed': 148971800,
   'quantum_spool_time': 5,
   'quantum_fuel_capacity': 645,
   'quantum_range': 65816326530.61224},
  'agility': {'pitch': 42,
   'yaw': 44,
   'roll': 145,
   'acceleration': {'main': 50.621,
    'retro'

In [11]:
data = data["data"]

In [41]:
from filters import fix_camelcase

output = ""

def oprint(text):
    global output
    output += text + "\n"

for item in data.items():
    match item:
        case "name", type_:
            oprint(f"# Ship Stats: {type_}")
        case "cargo_capacity", capacity:
            oprint(f"Cargo capacity: {capacity} SCU")
        case "sizes", {"length": l, "beam": w, "height": h}:
            oprint(f"Dimensions: {l}m length, {w}m width, {h}m height")
        case "emission", emission:
            oprint(f"Emissions: IR: {emission['ir']}, EM Idle: {emission['em_idle']}, EM Max: {emission['em_max']}")
        case "mass", mass:
            oprint(f"Mass: {mass} kg")
        case "vehicle_inventory", vehicle_inventory:
            oprint(f"Vehicle Inventory: {vehicle_inventory}")
        case "personal_inventory", personal_inventory:
            oprint(f"Personal Inventory: {personal_inventory}")
        case "crew", crew:
            crew_info = f"Crew: Skeleton: {crew['min']}"
            if crew['max'] is not None:
                crew_info += f", Max: {crew['max']}"
            if crew['weapon'] is not None:
                crew_info += f", Combat: {crew['weapon']}"
            if crew['operation'] is not None:
                crew_info += f", Operation: {crew['operation']}"
            oprint(crew_info)
        case "health", health:
            oprint(f"Health: {health}")
        case "shield_hp", shield_hp:
            oprint(f"Shield HP: {shield_hp} (From shield generator)")
        case "speed", speed:
            oprint((f"Speed: SCM: {speed['scm']} m/s, Max: {speed['max']} m/s, "
                       f"0 to SCM: {speed['zero_to_scm']} s, 0 to Max: {speed['zero_to_max']} s, "
                       f"SCM to 0: {speed['scm_to_zero']} s, Max to 0: {speed['max_to_zero']} s"))
        case "fuel", fuel:
            oprint((f"Fuel: Capacity: {fuel['capacity']}L, Intake Rate: {fuel['intake_rate']}L/s, "
                       f"Usage: Main: {fuel['usage']['main']}L\s, Maneuvering: {fuel['usage']['maneuvering']}L\s, "
                       f"Retro: {fuel['usage']['retro']}L\s, VTOL: {fuel['usage']['vtol']}L\s"))
        case "quantum", quantum:
            oprint((f"Quantum: Speed: {quantum['quantum_speed']}km/s, Spool Time: {quantum['quantum_spool_time']} s, "
                       f"Fuel Capacity: {quantum['quantum_fuel_capacity']}L, Range: {quantum['quantum_range']} m"))
        case "agility", agility:
            oprint((f"Agility: Pitch: {agility['pitch']}°/s, Yaw: {agility['yaw']}°/s, Roll: {agility['roll']}°/s, "
                       f"Acceleration: Main: {agility['acceleration']['main']} m/s², "
                       f"Retro: {agility['acceleration']['retro']} m/s², "
                       f"VTOL: {agility['acceleration']['vtol']} m/s², "
                       f"Maneuvering: {agility['acceleration']['maneuvering']} m/s²"))
        case "armor", armor:
            oprint(f"Armor: IR: {armor['signal_infrared']}, EM: {armor['signal_electromagnetic']}, Cross Section: {armor['signal_cross_section']}")
        case "foci", [focus]:
            oprint(f"Focus: {focus}")
        case "foci", foci:
            oprint(f"Foci: {', '.join(foci)}")
        case "type", ship_type:
            oprint(f"Role: {ship_type}")
        case "description", description:
            oprint(f"Description: {description}")
        case "size_class", size_class:
            oprint(f"Size Number: {size_class}")
        case "manufacturer", manufacturer:
            oprint(f"Manufacturer: {manufacturer['name']} ({manufacturer['code']})")
        case "insurance", insurance:  
            oprint((f"Insurance: Claim Time: {insurance['claim_time']}m, Expedite Time: {insurance['expedite_time']}m, "
                       f"Expedite Cost: {insurance['expedite_cost']} aUEC"))
        case "updated_at", updated_at:
            oprint(f"Updated At: {updated_at}")
        case "version", version:
            oprint(f"Version: {version}")
        case "production_status", production_status:
            oprint(f"Production Status: {production_status.replace('-', ' ').title()}")
        case "production_note", production_note if production_note != "None":
            oprint(f"Production Note: {production_note}")
        case "size", size:
            oprint(f"Size: {size.title()}")
        case "msrp", msrp:
            oprint(f"MSRP: ${msrp} USD")
        case "loaner", loaner:
            oprint(f"Loaner Ships: {', '.join(loaner) if loaner else 'None'}")
        case "hardpoints", components:
            oprint("## Components")
            complete_uuids = []
            for component in components:
                if "item" not in component:
                    continue
                count = lambda name: len([x for x in components if (x["type"] == name)])
                match component["item"]:
                    case {"name": name, "type": role, "size": size, "grade": grade, "class": class_, "uuid": uuid, "description": description} if uuid not in complete_uuids and description:
                        qty = count(role)
                        role = fix_camelcase(role)
                        s = "s" if qty > 1 else ""
                        oprint(f"{qty}x Size {size} {role}{s} ({class_} grade {grade})")
                        complete_uuids.append(uuid)
        case "slug", slug:
            # Ignored as it seems unimportant to an end-user
            ...
        case "class_name", class_name:
            # Ignored as it seems unimportant to an end-user
            ...
        case "uuid", uuid:
            # Ignored as it seems unimportant to an end-user
            ...
        case "id", id:
            # Ignored as it seems unimportant to an end-user
            ...
        case "chassis_id", chassis_id:
            # Ignored as it seems unimportant to an end-user
            ...
        case edge:
            print(f"Unhandled item: {edge}")

Unhandled item: ('shops', [{'uuid': 'f7b1f854-25f3-496c-9934-4776a34b1afd', 'name_raw': 'Astro Armada, Area 18', 'name': 'Astro Armada', 'position': 'Area 18', 'profit_margin': 50, 'link': 'https://api.star-citizen.wiki/api/v2/shops/f7b1f854-25f3-496c-9934-4776a34b1afd', 'version': '3.19.1-LIVE.8526421', 'items': [{'uuid': '831fdcfc-337a-4350-967d-e693d19efeee', 'name': 'Anvil C8X Pisces Expedition', 'type': 'Vehicle', 'sub_type': 'Vehicle_Spaceship', 'base_price': 406000, 'price_calculated': 406000, 'price_range': {'min': 406000, 'max': 406000}, 'base_price_offset': 0, 'max_discount': 0, 'max_premium': 0, 'inventory': 100, 'optimal_inventory': 100, 'max_inventory': 100, 'auto_restock': True, 'auto_consume': False, 'refresh_rate': 1, 'buyable': True, 'sellable': False, 'rentable': False, 'version': '3.19.1-LIVE.8526421'}]}])
Unhandled item: ('parts', [{'name': 'Body', 'display_name': 'Body', 'damage_max': 1500, 'children': [{'name': 'Canopy', 'display_name': 'Canopy', 'damage_max': 250

In [42]:
print(output)

# Ship Stats: C8X Pisces Expedition
Dimensions: 16m length, 12m width, 8m height
Emissions: IR: 6034, EM Idle: 4859, EM Max: 20278
Mass: 48143 kg
Cargo capacity: 4 SCU
Vehicle Inventory: 4
Personal Inventory: 0
Crew: Skeleton: 1, Combat: 0
Health: 3250
Shield HP: 1500 (From shield generator)
Speed: SCM: 142 m/s, Max: 1150 m/s, 0 to SCM: 2.805 s, 0 to Max: 22.717 s, SCM to 0: 28.979 s, Max to 0: 234.693 s
Fuel: Capacity: 522500L, Intake Rate: 12L/s, Usage: Main: 1L\s, Maneuvering: 14L\s, Retro: 0L\s, VTOL: 0L\s
Quantum: Speed: 148971800km/s, Spool Time: 5 s, Fuel Capacity: 645L, Range: 65816326530.61224 m
Agility: Pitch: 42°/s, Yaw: 44°/s, Roll: 145°/s, Acceleration: Main: 50.621 m/s², Retro: 4.9 m/s², VTOL: 0 m/s², Maneuvering: 95.86 m/s²
Focus: Exploration
Role: exploration
Description: Additional firepower turns the base Pisces into a highly capable mini-explorer. Ideal for inter-system travel or investigating tight spots inaccessible to larger ships, it’s a small ship ready for big 